In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce

DB Password: ········


In [5]:
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()
Control_BP_0 = pq.read_table('Cohorts/Control/Measurement/Control_BP_0.parquet').to_pandas()
Control_BP_1 = pq.read_table('Cohorts/Control/Measurement/Control_BP_50000.parquet').to_pandas()
Control_BP_2 = pq.read_table('Cohorts/Control/Measurement/Control_BP_100000.parquet').to_pandas()
Control_BP_3 = pq.read_table('Cohorts/Control/Measurement/Control_BP_150000.parquet').to_pandas()
Control_BP_4 = pq.read_table('Cohorts/Control/Measurement/Control_BP_200000.parquet').to_pandas()
Control_BP_5 = pq.read_table('Cohorts/Control/Measurement/Control_BP_250000.parquet').to_pandas()
Control_BP_6 = pq.read_table('Cohorts/Control/Measurement/Control_BP_300000.parquet').to_pandas()
Control_BP_7 = pq.read_table('Cohorts/Control/Measurement/Control_BP_350000.parquet').to_pandas()
Control_BP_8 = pq.read_table('Cohorts/Control/Measurement/Control_BP_400000.parquet').to_pandas()
Control_BP_9 = pq.read_table('Cohorts/Control/Measurement/Control_BP_450000.parquet').to_pandas()
Control_BP_10 = pq.read_table('Cohorts/Control/Measurement/Control_BP_500000.parquet').to_pandas()


#Cases = pq.read_table('Cohorts/ML/Cases_Lab_BMI_180_days_after').to_pandas()

In [19]:
Control_BP_10 = Control_BP_10.loc[Control_BP_10["context_name"]== "TDS"]

In [26]:
Control_BP = pd.DataFrame()

In [27]:
type(Control_BP)

pandas.core.frame.DataFrame

In [41]:
Control_BP = pd.concat([Control_BP, Control_BP_10])

In [42]:
Control_BP

,medical_record_number,age_in_days,time_of_day_key,context_name,context_procedure_code,numeric_value,unit_of_measure
0,1185340135,7434,821,TDS,SBP,104.0,mm Hg
3,2647419161,7579,710,TDS,DBP,72.0,mm Hg
6,2703000602,2776,1065,TDS,DBP,54.0,mm Hg
9,2198659975,7196,824,TDS,SBP,127.0,mm Hg
12,2315563257,4602,679,TDS,SBP,106.0,mm Hg
15,2654624367,7865,783,TDS,DBP,83.0,mm Hg
18,2714889200,6115,665,TDS,SBP,114.0,mm Hg
21,1949984269,6374,881,TDS,DBP,72.0,mm Hg
24,1152279404,5332,401,TDS,SBP,138.0,mm Hg
27,1692501912,6021,609,TDS,SBP,118.0,mm Hg


In [8]:
Case_BP_SBP = Case_BP.loc[Case_BP["context_procedure_code"] == "SBP"]

In [9]:
Case_BP_DBP = Case_BP.loc[Case_BP["context_procedure_code"] == "DBP"]

In [43]:
Control_BP_SBP = Control_BP.loc[Control_BP["context_procedure_code"] == "SBP"]

In [44]:
Control_BP_DBP = Control_BP.loc[Control_BP["context_procedure_code"] == "DBP"]

In [45]:
# Get min,max,mean and create a new dataframe with these three values from the output of the function filter_df_onset
def min_max_mean(df, lab_value_name):
    df_min = df.groupby([df["medical_record_number"]])['numeric_value'].min().to_frame()
    df_max = df.groupby([df["medical_record_number"]])['numeric_value'].max().to_frame()
    df_mean = df.groupby([df["medical_record_number"]])['numeric_value'].mean().to_frame()
    df_min.rename(columns={"numeric_value": lab_value_name+"_min"}, inplace=True)
    df_max.rename(columns={"numeric_value": lab_value_name+"_max"}, inplace=True)
    df_mean.rename(columns={"numeric_value": lab_value_name+"_mean"}, inplace=True)
    result = df_min.merge(df_max, left_index=True, right_index=True)
    result = result.merge(df_mean, left_index=True, right_index=True)
    return result

In [50]:
Control_SBP_min_max_mean = min_max_mean(Control_BP_SBP, "SBP")

In [53]:
Control_DBP_min_max_mean.to_parquet("Cohorts/Control/Measurement/Control_DBP_min_max_mean.parquet")

In [28]:
Case_DBP_filtered = Case_BP_DBP.merge(Case_filtered_15_540[["HT_Onset"]], left_on="medical_record_number", right_index=True, how="inner")

In [29]:
Case_DBP_filtered = Case_DBP_filtered.loc[(Case_DBP_filtered["HT_Onset"] - Case_DBP_filtered["age_in_days"]) > 180]

In [37]:
Case_SBP_min_max_mean = min_max_mean(Case_SBP_filtered, "SBP")

In [39]:
Case_SBP_min_max_mean.to_parquet("Cohorts/Case/Measurement/Case_SBP_min_max_mean.parquet")

In [32]:
Case_DBP_filtered.to_parquet("Cohorts/Case/Measurement/Case_DBP_180_days.parquet")

In [40]:
BMI = pq.read_table("Cohorts/Case/BMI_Cases_min_max_mean_360days_after_HT_onset.parquet").to_pandas()

In [46]:
pulse = pq.read_table("Cohorts/Case/Pulse_rate_Cases_min_max_mean_360days_after_HT_onset.parquet").to_pandas()

In [47]:
BMI_SBP = BMI.merge(Case_SBP_min_max_mean, left_index=True, right_index=True, how="inner")

In [49]:
BMI_SBP_Pulse = BMI_SBP.merge(pulse, left_index=True, right_index=True, how="inner")

In [53]:
Case_DBP_min_max_mean

,DBP_min,DBP_max,DBP_mean
medical_record_number,,,
1000012395,76.0,89.0,82.500000
1000154638,46.0,78.0,65.375000
1000227914,70.0,98.0,83.666667
1000508752,54.0,91.0,69.052632
1000519317,84.0,84.0,84.000000
1000560997,91.0,94.0,92.500000
1000604785,60.0,88.0,74.000000
1000614901,85.0,85.0,85.000000
1000628903,65.0,88.0,71.142857


In [55]:
BMI_BP_Pulse = BMI_SBP_Pulse.merge(Case_DBP_min_max_mean, right_index=True, left_index=True, how="inner")

In [58]:
BMI_BP_Pulse.to_parquet("Cohorts/ML/Cases_BMI_BP_Pulse.parquet")

In [63]:
Case_Labs

,CARBON_DIOXIDE_BLD_min,CARBON_DIOXIDE_BLD_max,CARBON_DIOXIDE_BLD_mean,UREA_NITROGEN-BLD_min,UREA_NITROGEN-BLD_max,UREA_NITROGEN-BLD_mean,CREATININE-SERUM_min,CREATININE-SERUM_max,CREATININE-SERUM_mean,HEMATOCRIT_min,HEMATOCRIT_max,HEMATOCRIT_mean,GLUCOSE_min,GLUCOSE_max,GLUCOSE_mean
medical_record_number,,,,,,,,,,,,,,,
1000096552,23.0,30.0,26.600000,11.0,29.0,17.600000,0.50,0.90,0.680000,31.2,43.7,35.720000,105.0,189.0,134.000000
1000173142,27.4,30.1,28.750000,19.0,23.0,21.000000,0.69,0.84,0.765000,37.0,41.4,39.400000,99.0,131.0,115.000000
1000227914,19.0,24.8,22.626667,9.0,16.0,12.133333,0.60,1.00,0.780000,35.8,47.7,43.682353,69.0,110.0,85.333333
1000271684,16.3,24.1,21.400000,51.0,70.0,60.250000,1.90,6.00,3.725000,22.0,38.4,26.829412,70.0,272.0,113.529412
1000273082,20.0,23.5,22.083333,3.0,12.0,7.666667,0.50,0.70,0.600000,32.5,35.4,33.866667,62.0,101.0,76.666667
1000434381,20.0,24.0,22.433333,9.0,10.0,9.333333,1.00,1.20,1.066667,38.0,48.8,41.990909,30.0,126.0,85.500000
1000546245,15.5,15.5,15.500000,17.0,17.0,17.000000,0.70,0.80,0.750000,35.6,37.0,36.300000,284.0,301.0,292.500000
1000604785,25.7,25.7,25.700000,23.0,23.0,23.000000,0.80,0.80,0.800000,40.2,40.2,40.200000,105.0,105.0,105.000000
1000628903,26.8,26.8,26.800000,51.0,51.0,51.000000,8.90,8.90,8.900000,43.0,43.0,43.000000,91.0,91.0,91.000000


In [67]:
Case_BMI_BP_Pulse_Labs_not_imputed = BMI_BP_Pulse.merge(Case_Labs, left_index=True, right_index=True, how="left")

In [70]:
Case_BMI_BP_Pulse_Labs_not_imputed.to_parquet("Cohorts/Case/Case_BMI_BP_Pulse_Labs_not_imputed.parquet")

In [31]:
Case_DBP_filtered.groupby([Case_DBP_filtered["medical_record_number"]])["age_in_days"].min().to_frame()

,age_in_days
medical_record_number,
1000012395,9834
1000154638,26393
1000227914,18611
1000508752,24740
1000519317,29599
1000560997,12731
1000604785,24386
1000614901,30352
1000628903,16284


In [14]:
Case_BP_SBP.to_parquet("Cohorts/Case/Measurement/Case_SBP.parquet")


In [25]:
Case_DBP_min_age = Case_BP_DBP.groupby([Case_BP_DBP["medical_record_number"]])["age_in_days"].min().to_frame()

In [27]:
Case_DBP_min_age = Case_filtered_15_540.merge(Case_DBP_min_age, left_index= True, right_index = True, how="inner")

In [33]:
c = Case_DBP_min_age.loc[Case_DBP_min_age["HT_Onset"] - Case_DBP_min_age["age_in_days"] > 180]

In [35]:
c

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,age_in_days
medical_record_number,,,,,,,,,,,,,
100067499,82,24103.0,NaN,24106.0,24103.0,Age_BP_condition,21049,3054.0,1951,White,Other,Male,22744
1008498682,87,23048.0,23112.0,23112.0,23048.0,Age_BP_condition,18738,4310.0,1948,Black Or African-American,Baptist,Male,22542
1018400572,155,9878.0,9895.0,9878.0,9878.0,Age_BP_condition,7519,2359.0,1987,White,Catholic,Female,8455
1019848184,383,18106.0,17818.0,17948.0,17818.0,Age_ICD_condition,12384,5434.0,1966,Ba,Baptist,Female,13495
1022736112,80,25425.0,25388.0,NaN,25388.0,Age_ICD_condition,23192,2196.0,1944,White,Unknown,Male,23961
1036599833,1664,5626.0,NaN,5623.0,5623.0,Age_MED_condition,4527,1096.0,1996,Asian Indian,Hindu,Female,4556
1036859119,349,26152.0,26626.0,26783.0,26152.0,Age_BP_condition,24959,1193.0,1940,White,Pt Declined,Female,25057
1038981872,103,NaN,19223.0,18436.0,18436.0,Age_MED_condition,15038,3398.0,1963,White,Unknown,Male,17783
1039156031,89,26224.0,NaN,NaN,26224.0,Age_BP_condition,24948,1276.0,1946,Unknown,Christian,Female,25578


In [34]:
Cases.merge(c, left_index=True, right_index=True, how="inner")

,Age_at_prediction,CARBON_DIOXIDE_BLD_max,CARBON_DIOXIDE_BLD_mean,CARBON_DIOXIDE_BLD_min,CREATININE-SERUM_max,CREATININE-SERUM_mean,CREATININE-SERUM_min,GLUCOSE_max,GLUCOSE_mean,GLUCOSE_min,...,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth_y,race,religion,gender,age_in_days
medical_record_number,,,,,,,,,,,,,,,,,,,,,
1008498682,22868.0,27.9,25.900000,22.6,1.20,1.100000,1.00,148.0,114.666667,59.0,...,23112.0,23048.0,Age_BP_condition,18738,4310.0,1948,Black Or African-American,Baptist,Male,22542
1022736112,25208.0,27.0,27.000000,27.0,0.80,0.800000,0.80,80.0,80.000000,80.0,...,NaN,25388.0,Age_ICD_condition,23192,2196.0,1944,White,Unknown,Male,23961
1069583639,18944.0,28.3,25.550000,23.3,1.11,0.855000,0.75,93.0,79.750000,71.0,...,19822.0,19124.0,Age_ICD_condition,17522,1602.0,1964,Other,PT Declined,Female,17583
1078126736,25703.0,26.2,26.200000,26.2,0.70,0.700000,0.70,78.0,78.000000,78.0,...,25883.0,25883.0,Age_BP_condition,20822,5061.0,1947,White,Jewish,Female,25024
1099400909,26101.0,28.6,28.600000,28.6,0.80,0.800000,0.80,91.0,91.000000,91.0,...,26283.0,26281.0,Age_ICD_condition,21871,4410.0,1943,White,Jewish,Female,21871
1126366307,21323.0,29.3,28.011111,25.4,1.51,1.384444,1.30,90.0,81.111111,74.0,...,22557.0,21503.0,Age_BP_condition,18416,3087.0,1954,White,Christian,Male,20788
1162872476,19528.0,30.0,27.616667,24.8,0.90,0.833333,0.80,279.0,133.500000,63.0,...,NaN,19708.0,Age_ICD_condition,16905,2803.0,1957,Ba,Catholic,Female,18802
1163814305,18387.0,30.6,27.400000,25.3,1.60,1.137143,0.86,77.0,72.800000,69.0,...,NaN,18567.0,Age_BP_condition,13819,4748.0,1963,Ba,,Female,17483
1163982403,11864.0,23.2,23.200000,23.2,0.60,0.600000,0.60,167.0,117.000000,87.0,...,NaN,12044.0,Age_BP_condition,10732,1312.0,1981,White,Christian,Female,10732
